In [1]:
!pip install langchain-gigachat

In [2]:
from langchain_gigachat.chat_models import GigaChat

In [ ]:
llm = GigaChat(
#             model="GigaChat-Max",
              credentials="API_key",
              scope="GIGACHAT_API_PERS"
              verify_ssl_certs=False,
              )


Getting the Bot Token. We need a Telegram bot. Main steps:
- Find the @BotFather account in Telegram
- Send the command /newbot
- Follow BotFather’s instructions to choose a name (any) and a username (unique, must include “bot” in it)
- After creating the bot, BotFather will give you a token to access the Telegram Bot API

In [4]:
!pip install -q pyTelegramBotAPI

Working with Memory (Context Window) in LangChain  
  

Language models don’t have memory by default. They simply generate new text based on the prompt they receive. The prompt may contain the entire dialogue history starting from the system message or the first user message. But as the dialogue grows, each new user query is preceded by an increasingly long conversation history. This leads to:  
- High token usage  
- Risk of exceeding the model’s context window  

To avoid this and optimize token usage, LangChain provides different **types of meory**:  

- **`ConversationBufferMemory`** – simplest option, stores and passes all dialogue messages.  
- **`ConversationBufferWindowMemory`** – similar to above, but keeps only the last *k* message pairs (user + model), acting as a sliding window.  
- **`ConversationEntityMemory`** – stores both dialogue history and extracted structured entities (names, objects, etc.) alongside unstructured text.  
- **`ConversationKGMemory`** – keeps memory as a **knowledge graph**.  
- **`ConversationSummaryMemory`** – keeps a **summary of the dialogue** instead of full messages. The model periodically updates the summary with new details.  
- **`ConversationSummaryBufferMemory`** – combines a limited message history with an overall summary of the conversation.  
- **`ConversationTokenBufferMemory`** – similar to window memory, but limits based on **number of tokens** instead of messages.  
- **`VectorStoreRetrieverMemory`** – stores dialogue in a vector database and retrieves only **semantically relevant** messages to include in context.  

👉 In practice:  
- Use **Buffer/Window memory** for simple chatbots.  
- Use **Summary memory** to scale long conversations.  
- Use **Vector memory** when retrieval of specific past facts is needed.  


ConversationBufferMemory

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [2]:
conversation1 = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [3]:
conversation1.predict(input="Hello! My name is John. I’m currently building a chatbot with LangChain and Python.")
conversation1.predict(input="Which large language model is considered the most advanced right now?")
conversation1.predict(input="I’d like to use a language model for free and write in English.")
conversation1.predict(input="What’s my name and what project am I working on?")

ConversationEntityMemory

In [8]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.memory import ConversationEntityMemory

In [4]:
conversation2 = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [5]:
conversation2.predict(input="Hello! My name is John. I’m currently building a chatbot with LangChain and Python.")
conversation2.predict(input="Which large language model is considered the most advanced right now?")
conversation2.predict(input="I’d like to use a language model for free and write in English.")
conversation2.predict(input="What’s my name and what project am I working on?")

In [6]:
conversation2.memory.entity_store.store

Create a bot

In [12]:
user_conversations = {}

In [ ]:
import telebot
from time import sleep

bot = telebot.TeleBot('Bot_API')

In [14]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory())

In [15]:
@bot.message_handler(content_types=['audio',
                                    'video',
                                    'document',
                                    'photo',
                                    'sticker',
                                    'voice',
                                    'location',
                                    'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, "I don’t work with non-text messages!")

@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id
    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationBufferMemory()
    conversation.memory = user_conversations[user_id]
    response = conversation.predict(input=message.text)
    bot.send_message(user_id, "I won’t ignore your message, but I cannot avoid responding.")
    sleep(2)

In [17]:
template = '''
You are a chatbot answering questions as a consultant for SoftAI Solutions.  
Your reply must be no longer than 20 words. Use only the data below:  
SoftAI Solutions develops AI software and analytics tools.  
Phone: +1 (555) 123-4567. Email: support@softai.com.  
We are based in New York, 45 Innovation Street.  
Products: corporate AI platforms, chatbots, and data analytics.  
Purchases: via website softai.com or by phone.  
Refunds: available within 14 days if the product is intact.  
Complaints and feedback: send to feedback@softai.com.  
For greetings reply: "How can I help? I’m here to answer questions about SoftAI Solutions."  
For any unrelated queries reply: "Sorry, I only provide information about SoftAI Solutions."  

\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:
'''

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)
conversation.prompt.template = template

In [7]:
conversation.prompt

In [8]:
bot.polling(none_stop=True)